# Fractional hybridization of Carbon atoms

This Jupyter Notebook illustrates how to compute hybridization indexes of C atoms from xyz structure files of molecular systems using few lines of Python code. Some of the functions defined below are re-usable for other purposes. 

First, let review some basic concepts about hybridization. Let consider a C atom using some kind of $sp^{\tau}$ hybrid orbitals to form $\sigma$ bonds with the neighboring substituents. The (unnormalized) hybrids can be represented in the form:


\begin{equation}
|\psi_i \rangle = |s\rangle + \sqrt{\tau_i}|p\rangle
\end{equation}

where $|s\rangle$ describes a (total-symmetric) $s$ orbital and $|p\rangle$ a directional $p$-orbital, while
$\tau_i$ is the fraction of $p$ character of the hybrid, a.k.a. the *hybridization index*. Notice that $|s\rangle$, $|p\rangle$ represent a basis in which the hybrid orbital is expanded; $|p\rangle$, being a directional orbital, can be further expanded in the local basis $|p_x\rangle, |p_y\rangle, |p_z\rangle$. Assuming
orthogonality w.r.t. to the scalar product defined in the Hilbert space $H$ spanned by the hybrids, we have:

\begin{equation}
\langle \psi_i | \psi_j \rangle = 1 + \sqrt{\tau_i \tau_j}\cos(\theta_{ij}) = \delta_{ij}
\end{equation}

In the hypothesis of non-bent bonds, $\theta_{ij}$ are the corresponding bond angles. Therefore, the hybridization indexes $\tau_i$ can be computed directly from the geometry around the C atom as:

\begin{equation}
\boxed{\cos(\theta_{ij}) = -\frac{1}{\sqrt{\tau_i\tau_j}}}
\end{equation}

The above equation defines a system of three equations in three unknowns (namely $\tau_i$), which can be easily solved to give:

\begin{equation}
\boxed{\tau_i = \frac{\cos(\theta_{jk})}{\cos(\theta_{ij})\cos(\theta_{ik})}}
\end{equation}

with $i\neq j \neq k$. Notice that for the $\pi$-like orbital we may write:

\begin{equation}
|\psi_{\pi}\rangle = |s\rangle + \sqrt{\lambda} |p\rangle
\end{equation}

thus defining three additional equations of the same type as above, involving the angles $(\theta_{1\pi}, \theta_{2\pi}, \theta_{3\pi})$. The problem is only apparently overdetermined because the orientation of the fourth orbital actually follows from orthogonality once the orientation of the first three is given (a
consequence of having a 4-dimensional function space in 3-dimensional Euclidean space). The hybridization index   for the $\pi$-like orbital can thus be obtained by the conservation of $p$ (or $s$) weights upon hybridization. The latter simply follows from the condition:

\begin{equation}
\sum_{i=1}^3 ||\psi_i||^2 + ||\psi_{\pi}||^2 = 1
\end{equation}

which leads to:

\begin{equation}
\boxed{ \lambda = \frac{1}{1-\sum_i(1+\tau_i)^{-1}} -1 }
\end{equation}

which is singular ($\lambda \to + \infty$) for co-planar $\sigma$-bonds, i.e. when $\tau_i = 2$, for $i = 1{-}3$.

We can use the above derived equations to compute the hybridization indexes of C atoms of a given molecule from its geometrical structure as given by the xyz file. The steps required for such computation are the following:

1. Read the xyz file
2. For each atom, find its nearest-neighbors (i.e. its substituents)
3. Compute the corresponding bond angles
4. Compute the hybridization indexes from the bond angles
5. Use the conservation law to compute the hybridization index of the $\pi$-like orbital



## Read the xyz file

With the following code, the xyz file of the molecule is read and the corresponding atom coordinates are stored into a list. We remind that the typical structure of an xyz file is the following: a first row specifying the number of atoms ($N_{at}$) in the structure, a second row specifying the title (optional, it can be left blank) and then a list of atom labels and coordinates (in the format "Label x y z").

In [6]:
# Import relevant packages:

from collections import Counter
from collections import defaultdict
from ssl import HAS_TLSv1_1
import sys
import numpy as np
import math
import matplotlib.pyplot as plt

# Defining function read_xyz() that reads a xyz file (filename)
def read_xyz(filename):
    """This function reads atom labels and coordinates from a xyz file.
    
    Args:
        filename (string): the name of the xyz file
    
    Returns:
        atoms (list): a list of atom labels 
        coordinates (list): a list of [x,y,z] coordinates 
    
    Raises:
        ValueError: the function raises an error if it reads a number of atoms different from what 
            specified by the header of the xyz file.
    """

    atoms = []
    coordinates = []

    xyz_file = open(filename)
    
    # Reading number of atoms 
    n_atoms = int(xyz_file.readline())

    # Reading title
    title = xyz_file.readline()

    # Reading atom labels and xyz coordinates 
    for line in xyz_file:
        atom, x, y, z = line.split()
        atoms.append(atom)
        coordinates.append([float(x),float(y), float(z)])

    xyz_file.close()

    # Check if number of atoms and number of coordinates match 
    if n_atoms != len(coordinates):
        raise ValueError("File says %d atoms but read %d points." % (n_atoms, len(coordinates)))

    return atoms, coordinates

# Reading filename from stdin
file = input("xyz? ")

atoms, coordinates = read_xyz(file)

xyz? coronene.xyz


Once we have executed the `read_xyz()` function, we can print some information about the molecule, such as the total number of atoms and the number of C atoms. Remind that this notebook and the corresponding Python code has been thought to illustrate the computation of the hybridization indexes of C atoms in a carbon-based molecular systems (hydrocarbons, aromatic molecules, polyciclic aromatic hydrocarbons, etc.). Therefore, we exit the code if the xyz does not contain C atoms. One can modify and re-use the code if needed for other purposes. 

In [7]:
# Molecule info:
n_atoms = len(atoms)

atom_counter = Counter(atoms)
n_carbons = atom_counter.get("C", 0)
if n_carbons == 0:
    sys.exit("ERROR! No C atoms in the xyz file!")

print("")
print('#'*80)
print("")
print("filename:                %s" % file)
print("number of atoms:         %d" % n_atoms)
print("number of distinct C:    %d" % n_carbons)
print("")
print('#'*80)


################################################################################

filename:                coronene.xyz
number of atoms:         36
number of distinct C:    24

################################################################################


## Finding nearest-neighbors 

To compute the relevant bond angles for the computation of hybridization indexes, we first need to identify  nearest-neighbors of C atoms. Here, we are interested in just the first 3 nearest-neighbors, since it assumed we are dealing with $\pi$-conjugated systems.

In [8]:
# Converting coordinates to NumPy array
coord_np = np.array(coordinates)

def nn(i, coord):
    """ This function determines the first three nearest-neighbors of an atom from the xyz file.

    Args:
        i (integer): index of the target atom 
        coord (NumPy array): array specifying xyz coordinates of atoms in the molecule

    Returns:
        nn_indx (list): list of nearest-neighbors' indexes
        nn_dist (list): list of corresponding distances from target atom (i.e. bond distances)
    """

    # Finding number of atoms
    natom, ncoord = coord.shape

    # Computing Euclidean distances 
    dist_list = np.array([np.linalg.norm(coord[i]-coord[j]) for j in range(natom) ])
    nn_indx = list(np.argsort(dist_list)[1:4])
    nn_dist = dist_list[nn_indx]

    return nn_indx, nn_dist

As an use example of the above function, let find the nearest-neighbors of the atom of label 1 and the corresponding bond distances:

In [11]:
# Finding nearest-neighbors of atom 1 and the corresponding bond distances:
nn_1, nn_dist_1 = nn(1, coord_np)

print(f"Nearest-neighbors of Atom 1 ( a {atoms[1]} atom) are: {nn_1}.")
print(f"Corresponding bond distances in Angstrom: {nn_dist_1}")

Nearest-neighbors of Atom 1 ( a C atom) are: [26, 0, 2].
Corresponding bond distances in Angstrom: [1.08689772 1.37369383 1.4251057 ]


## Computing bond angles

We are now in position to compute the bond angles for any atoms in the molecule. Bond angles are simply given by the dot product between (normalized) bond vectors $v_i$

\begin{equation}
\cos(\theta_{ij}) = \frac{ v_i \cdot v_j}{||v_i|| ||v_j||}
\end{equation}

We can again test the function computing bond angles of Atom 1.

In [ ]:
def angle(i, coord_np):
    '''This function computes bond angles of an atom of index i.

    Args:
        i (integer) : index of the target atom
        coord_np (Numpy array) : array specifying xyz coordinates of atoms in the molecule
    
    Returns:
        dot_list (list) : a list of bond vectors dot products 
        angles : a list of bond angles
    '''
    
    # Getting first three-nearest neighbors calling function nn():
    nn_i, dist_nn_i = nn(i, coord_np)
    
    # Building vectors connecting atom i to its nearest neighbors and normalizing them:
    vector_nn = np.array([coord_np[i] - coord_np[nn_i[j]] for j in range(3)])
    unit_vector_nn = np.array([vector_nn[i]/np.linalg.norm(vector_nn[i]) for i in range(3)])
    
    # Computing dot products between units vetors:
    dot_prods = []
    prod1 = np.dot(unit_vector_nn[0], unit_vector_nn[1])
    prod2 = np.dot(unit_vector_nn[1], unit_vector_nn[2])
    prod3 = np.dot(unit_vector_nn[0], unit_vector_nn[2])
    dot_prods.extend([prod1, prod2, prod3])

    dot_list = list(dot_prods)
    angles = [math.degrees(np.arccos(x)) for x in dot_prods]

    return dot_list, angles

# Testing the function against atom 1:
atom1_dot_prods, atom1_angles = angle()

print(f"Bond angles of Atom 1 are {})